### Web thứ nhất: https://careerviet.vn
Các công việc trong ngành IT được lấy ra trong web:
- Data Engineer
- Data Analyst
- Data Scientist

In [4]:
import requests
from bs4 import BeautifulSoup
import csv

def fetch_job_data(url):
    # Header để giả lập trình duyệt
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
    }

    # Gửi yêu cầu đến trang web
    response = requests.get(url, headers=headers)

    job_list = []

    # Kiểm tra trạng thái phản hồi
    if response.status_code == 200:
        # Phân tích HTML bằng BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser", from_encoding='utf-8')
        
        # Tìm tất cả các công việc
        jobs = soup.find_all("div", class_="job-item")
        for job in jobs:
            job_name_tag = job.find("a", class_="job_link")
            job_name = job_name_tag.text.strip() if job_name_tag else "N/A" # Tiêu đề công việc
            job_link = job_name_tag['href'] if job_name_tag and 'href' in job_name_tag.attrs else "N/A" # Link chi tiết công việc
            company = job.find("a", class_="company-name").text.strip() if job.find("a", class_="company-name") else "N/A" # Tên công ty
            location = job.find("ul").text.strip() if job.find("ul") else "N/A" # Vị trí công ty
            
            # Truy cập trang chi tiết công việc để lấy thông tin lương, kinh nghiệm, cấp bậc, mô tả công việc và yêu cầu công việc
            salary = "N/A"
            experience = "N/A"
            position = "N/A"
            job_requirements = "N/A"  # Thông tin mặc định nếu không tìm thấy dữ liệu
            if job_link != "N/A":
                detail_response = requests.get(job_link, headers=headers)
                if detail_response.status_code == 200:
                    detail_soup = BeautifulSoup(detail_response.text, "html.parser", from_encoding='utf-8')
                    li_tags = detail_soup.find_all("li")
                    for li in li_tags:
                        # Từ thẻ li, tìm thông tin về kinh nghiệm yêu cầu và lương trong thẻ strong
                        if li.find("strong") and "Kinh nghiệm" in li.find("strong").text:
                            experience_tag = li.find("p")
                            if experience_tag:
                                experience = " ".join(experience_tag.text.split())
                        if li.find("strong") and "Lương" in li.find("strong").text:
                            salary_tag = li.find("p")
                            if salary_tag:
                                salary = " ".join(salary_tag.text.split())
                    
                    # Lấy thông tin yêu cầu công việc
                    job_requirements_sections = detail_soup.find_all("h2", string="Mô tả Công việc")
                    for section in job_requirements_sections:
                        job_requirements_div = section.find_next("div", class_="detail-row reset-bullet")
                        if job_requirements_div:
                            job_requirements = " ".join(job_requirements_div.text.split())

            # Lưu dữ liệu công việc
            job_list.append({
                "Job Name": job_name,
                "Job Link": job_link,
                "Company": company,
                "Location": location,
                "Salary": salary,
                "Experience": experience,
                "Job Requirements": job_requirements
            })
    else:
        print(f"Failed to fetch the page. Status code: {response.status_code}") # In ra lỗi nếu không truy cập được url

    return job_list

# URLs của các trang web
urls = [
    "https://careerviet.vn/viec-lam/Data-Engineer-k-vi.html",
    "https://careerviet.vn/viec-lam/Data-Analyst-k-vi.html",
    "https://careerviet.vn/viec-lam/Data-Scientist-k-vi.html"
]

# Tạo danh sách tất cả các công việc từ các trang web
all_jobs = []
for url in urls:
    all_jobs.extend(fetch_job_data(url))

# Lưu dữ liệu vào file CSV
with open('jobs.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Job Name", "Job Link", "Company", "Location", "Salary", "Experience", "Job Requirements"])
    for job in all_jobs:
        writer.writerow([job['Job Name'], job['Job Link'], job['Company'], job['Location'], job['Salary'], job['Experience'], job['Job Requirements']])

print("Dữ liệu đã được lưu vào file jobs.csv")


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


Dữ liệu đã được lưu vào file jobs.csv


### Web thứ hai: https://123job.vn/
Các công việc trong ngành IT được lấy trong web:
- Full Stack Developer
- Front End Developer
- Software Tester

In [5]:
import requests
from bs4 import BeautifulSoup
import csv
import os

def fetch_job_data(url):
    # Tạo headers để giả lập trình duyệt
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
    }

    # Gửi yêu cầu HTTP GET
    response = requests.get(url, headers=headers)

    # Kiểm tra nếu yêu cầu thành công
    if response.status_code == 200:
        # Phân tích nội dung HTML của trang
        soup = BeautifulSoup(response.text, 'html.parser')
    
        # Lấy thông tin công việc
        jobs = soup.find_all('div', class_='job__list-item-content')  # CSS selector tùy thuộc vào cấu trúc trang web
        job_list = []
    
        for job in jobs:
            job_name_tag = job.find('a')  # Lấy tiêu đề công việc
            job_name = job_name_tag.text.strip() if job_name_tag else "N/A" 
            job_link = job_name_tag['href'] if job_name_tag and 'href' in job_name_tag.attrs else "N/A" # Link chi tiết công việc
            # Lấy tên công ty
            company = job.find('div', class_='job__list-item-company text-collapse').text.strip() if job.find('div', class_='job__list-item-company text-collapse') else 'N/A'
            location = job.find('div', class_='address').text.strip() if job.find('div', class_='address') else 'N/A'  # Lấy địa điểm
            salary = job.find('div', class_='salary').text.strip() if job.find('div', class_='salary') else 'N/A'  # Lấy mức lương

            # Truy cập vào link chi tiết công việc để lấy thêm dữ liệu
            job_description = 'N/A'
            job_requirements = 'N/A'
            experience = 'N/A' # Mặc định nếu không tìm thấy dữ liệu

            # Truy cập trang chi tiết nếu có link
            if job_link != 'N/A':
                detail_response = requests.get(job_link, headers=headers)
                if detail_response.status_code == 200:
                    detail_soup = BeautifulSoup(detail_response.text, 'html.parser', from_encoding='utf-8') # Sử dụng BeautifulSoup để truy cập các link chi tiết công việc

                    # Tìm thông tin về yêu câu công việc
                    content_groups = detail_soup.find_all('div', class_='content-group')
                    for group in content_groups:
                        title = group.find('h2', class_='content-group__title')
                        if title and "Yêu cầu công việc" in title.text:
                            requirements_div = group.find('div', class_='content-group__content')
                            job_requirements = requirements_div.get_text(separator='\n').strip() if requirements_div else 'N/A'

                    # Tìm thông tin về kinh nghiệm yêu cầu
                    attr_item_tags = detail_soup.find_all('div', class_='attr-item')
                    for attr_item in attr_item_tags:
                        text_mb1_div = attr_item.find('div', class_='text mb-1')
                        if text_mb1_div and "Kinh nghiệm yêu cầu" in text_mb1_div.text:
                            experience_div = attr_item.find('div', class_='value text-bold text-center')
                            experience = experience_div.text.strip() if experience_div else 'N/A'
        
            # Lưu dữ liệu công việc
            job_list.append({
                "Job Name": job_name,
                "Job Link": job_link,
                "Company": company,
                "Location": location,
                "Salary": salary,
                "Experience": experience,
                "Job Requirements": job_requirements
            })
    else:
        print(f"Yêu cầu không thành công, mã trạng thái: {response.status_code}") # In ra lỗi nếu không truy cập được url

    return job_list

# URL các trang web
urls = [
    "https://123job.vn/tuyen-dung?s=0&sort=top_related&q=Fullstack+Developer&l=",
    "https://123job.vn/tuyen-dung?label=&s=0&jt=&level=&exp=&cat=&date=&cat_name=&benefit=&postType=&size=&contentHas=&sort=top_related&q=Frontend+Developer&l=",
    "https://123job.vn/tuyen-dung?s=0&sort=top_related&q=Software+Tester&l=%26quot%3B"
]

# Tạo danh sách tất cả các công việc từ các trang web
all_jobs = []
for url in urls:
    all_jobs.extend(fetch_job_data(url))

# Kiểm tra nếu file CSV đã tồn tại
file_exists = os.path.isfile('jobs.csv')

# Lưu dữ liệu vào file CSV
with open('jobs.csv', mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Ghi tiêu đề chỉ nếu file chưa tồn tại
    if not file_exists:
        writer.writerow(["Job Name", "Job Link", "Company", "Location", "Salary", "Experience", "Job Requirements"])
    for job in all_jobs:
        writer.writerow([job['Job Name'], job['Job Link'], job['Company'], job['Location'], job['Salary'], job['Experience'], job['Job Requirements']])

print("Dữ liệu mới đã được thêm vào file jobs.csv")


Dữ liệu mới đã được thêm vào file jobs.csv


In [6]:
import pandas as pd
data = pd.read_csv('jobs.csv')
data

,Job Name,Job Link,Company,Location,Salary,Experience,Job Requirements
0,DATA ENGINEER,https://careerviet.vn/vi/tim-viec-lam/data-eng...,CÔNG TY TNHH GLOBAL COMMERCIAL PARTNER,Hồ Chí Minh,12 Tr - 15 Tr VND,2 - 3 Năm,Yêu Cầu Công Việc Kinh nghiệm và kiến thức chu...
1,Data Engineer,https://careerviet.vn/vi/tim-viec-lam/data-eng...,Công ty CP Truyền thông và Công nghệ ICOMM Việ...,Hà Nội,Lên đến 50 Tr VND,NaN,Yêu Cầu Công Việc - Có kiến thức & kỹ năng thi...
2,DATA ENGINEER,https://careerviet.vn/vi/tim-viec-lam/data-eng...,CÔNG TY TNHH CÔNG NGHỆ B2B,Hồ Chí Minh,Cạnh tranh,3 - 8 Năm,Yêu Cầu Công Việc Kiến Thức:· Kiến thức về các...
3,Data Engineer,https://careerviet.vn/vi/tim-viec-lam/data-eng...,Huhtamaki (Vietnam) Ltd.,Bình Dương,Cạnh tranh,Chưa có kinh nghiệm,Yêu Cầu Công Việc Qualification: University En...
4,Senior Data Engineer,https://careerviet.vn/vi/tim-viec-lam/senior-d...,CÔNG TY CỔ PHẦN Z HOLDING,Hà Nội,20 Tr - 30 Tr VND,3 - 4 Năm,Yêu Cầu Công Việc - Educational Background: Ba...
...,...,...,...,...,...,...,...
139,[Automation/Manual] Software Tester - Up to 1500$,https://123job.vn/viec-lam/automation-manual-s...,Breadstack technologies inc,Hà Nội,1500 usd,Cập nhật,Yêu cầu công việc\nMust to have:\nExperience w...
140,[Hồ Chí Minh] Cần tuyển software tester mức lư...,https://123job.vn/viec-lam/ho-chi-minh-can-tuy...,Công ty phát triển phần mềm,"Quận 1, Hồ Chí Minh",20 triệu,2 năm kinh nghiệm,NaN
141,Senior Software Tester QA/QC/Manualupto 1200,https://123job.vn/viec-lam/senior-software-tes...,Solashi vietnam,Hà Nội,Đang cập nhật,Cập nhật,Yêu cầu công việc\nĐộ tuổi: từ 25- 30 tuổi\nHi...
142,Software Tester (Manual/Automation/QA QC),https://123job.vn/viec-lam/software-tester-man...,Grapecity,Hà Nội,Đang cập nhật,Cập nhật,Yêu cầu công việc\nProficiency in or Automatio...
